## Setting up the environment

In [ ]:
%%capture
files = !ls
files = [f.split("  ") for f in files][0]

isFRIQML = 'fri_qml' in files
isFRIQMLPath = isFRIQML and "setup.py" in files

# Clone the entire repo. Only run once!
if not isFRIQML:
  !git clone -l -s https://github.com/znajob/fri_qml.git fri_qml

if not isFRIQMLPath:
  %cd fri_qml

!git pull
!pip install -e .

In [ ]:
# MAIN IMPORTS
import logging
logging.basicConfig(level=logging.DEBUG)
import pennylane as qml
from pennylane import numpy as np
from friqml.visualisation import plot_quantum_state, plot_histogram
from friqml.utils import eps, random_state_normalized, random_state_unnormalized
from functools import partial

DEBUG:matplotlib:matplotlib data path: c:\ProgramData\anaconda3\envs\Quantum_ML\Lib\site-packages\matplotlib\mpl-data
DEBUG:matplotlib:CONFIGDIR=C:\Users\tomaz\.matplotlib
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is win32
DEBUG:matplotlib:CACHEDIR=C:\Users\tomaz\.matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from C:\Users\tomaz\.matplotlib\fontlist-v390.json


When solving the exercises refer to the [PennyLane documentation](https://pennylane.readthedocs.io/en/stable/).

## Quantum states

### Exercise 1

A quantum state is a pure quantum probability. A qubit state is a pure quantum probability over two values. A major difference with classical probability vector ($\vec{p}$ is real $L_1$ normalised) is that the entries are complex numbers and the normalization is in the $L_2$ norm. Create a function `is_quantum_state` that checks whether a vector is a valid quantum state. The input is a numpy array and the output should be boolean.

In [ ]:
###
### YOUR CODE HERE
###

def is_quantum_state(x):
  return np.allclose(np.linalg.norm(x),1)


In [ ]:
# TESTS
print(is_quantum_state(np.array([1/np.sqrt(2), 1/np.sqrt(2)])))
print(is_quantum_state(np.array([-1/np.sqrt(2), 1/np.sqrt(2)])))
print(is_quantum_state(np.array([-1/3, 2*np.sqrt(2)/3])))
print(is_quantum_state(np.array([-1j/3, 2*np.sqrt(2)/3])))
print(is_quantum_state(random_state_normalized(n=7)))
print(not is_quantum_state(random_state_unnormalized(n=6)))
print(not is_quantum_state(np.array([0.2, 0.8])))

True
True
True
True
True
True
True


### Exercise 2
Quantum circuit starts with a well defined state. In the qubit case, which we will consider, the initial state is a product state with all qubits in the state $|0\rangle$. Create a function `circuit` with 2 qubits in PennyLane and output the expectation values of the operator $A=\sigma_1^{z}+\sigma_2^z$.

In [ ]:
# DEVICE
dev = qml.device('default.qubit', wires=2, shots=None)

In [ ]:
###
### YOUR CODE HERE
###

@qml.qnode(dev)
def circuit():
  return qml.expval(qml.PauliZ(0)+qml.PauliZ(1))


In [ ]:
# TESTS
mz = circuit()
print(mz==2)
print((dev.state==np.array([1,0,0,0])).all())

### Exercise 3
A qubit can be conveniently represented in a bloch sphere. Create function `circuit` that rotates the initial qubit around the $y$ axis for angle `theta`, then rotates the qubit around the `z` axis for angle `phi`. and finally returns the expectation values of observable (random variable) $A=\sigma_x$.  

In [ ]:
# DEVICE
dev = qml.device('default.qubit', wires=1, shots=None)

In [ ]:
###
### YOUR CODE HERE
###
@qml.qnode(dev)
def circuit(theta,phi):
  qml.RY(theta,wires=0)
  qml.RZ(phi,wires=0)
  return qml.expval(qml.PauliX(0))

In [ ]:
# TESTS
phi=2*np.pi*np.random.rand()
theta=2*np.pi*np.random.rand()
circuit(phi,theta)
print(circuit(phi,theta)-np.cos(phi)*np.sin(theta)<eps)

**Note:** We can conveniently draw our circuit with `qml.draw(circuit)(*circuit_args)` functionallity which represents our circuit as a string. If you wand a nicer outpu you can use also the `qml.drawer` module.

In [ ]:
print(qml.draw(circuit)(np.pi/4, np.pi/2))

In [ ]:
qml.drawer.use_style('black_white')
fig, ax = qml.draw_mpl(circuit)(1.2345, 1.2345)

### Exercise 4
Since any observable quantity is a function of $\rho=|\psi⟩⟨\psi|$ all states/vectors $|\psi(\varphi)⟩$ of the form $|\psi(\varphi)⟩=\exp ({\rm i}\varphi)|\psi⟩$, where $\varphi\in[0,2\pi]$, represent the same quantum probability. Write a function `compare_states` with two inputs `psi1` and `psi2`. The function should return `True` if the quantum states represent the same system and `False` otherwise.

In [ ]:
# DEVICE
dev = qml.device('default.qubit', wires=1, shots=None)

In [ ]:
###
### YOUR CODE HERE
###

def compare_states(psi1,psi2):
  overlap = np.abs(np.vdot(psi1,psi2))
  return np.isclose(1,overlap)


In [ ]:
# TESTS
psi1 = random_state_normalized(4)
psi2 = random_state_normalized(4)
psi3 = psi1*np.exp(1j*np.random.rand()*np.pi*2)

print(compare_states(psi1,psi1))
print(compare_states(psi2,psi2))
print(compare_states(psi1,psi3))
print(not compare_states(psi1,psi2))

## Entanglement

### Exercise 1
We now consider a two qubit example. A two qubit state $|\psi⟩$ is a product state if it can be written as a tensor product of two one qubit states $|\psi⟩=|\psi_1⟩⊗|\psi_2⟩$. In the opposite case we call it entangled. Write a function `is_entangled` that recieves a two qubit state represented by a numpy array and returns `True` if the state is entangled and `False` otherwise.

In [ ]:
###
### YOUR CODE HERE
###

def is_entangled(psi):
  # rho = np.einsum("i,j->ij",np.conj(psi),psi)
  rho = np.outer(np.conj(psi),psi)
  rho1 = rho[:2,:2]+rho[2:,2:]
  val, _ = np.linalg.eigh(rho1)
  return  not np.allclose(val[0],0)

In [ ]:
psi_rand4 = random_state_normalized(4)
psi_prod2 = np.kron(random_state_normalized(2),random_state_normalized(2))
print(is_entangled(psi_rand4))
print(not is_entangled(psi_prod2))


**Note:** Since product states are very unlikely a random state will be entangled with probability $1-\epsilon$.

### Exercise 2
Write a two qubit function `circuit` that creates an entengled state $\phi^-=\frac{1}{\sqrt{2}}(|00⟩-|11⟩)$ and returns the expectation values of observables $A_1=\sigma^{\rm z}⊗\mathbb{I}_2$, $A_2=\mathbb{I}_2\otimes\sigma^{\rm z}$, and $A_3=\sigma^{\rm z}_1\otimes \sigma^{\rm z}_2$

In [ ]:
# DEVICE
dev = qml.device('default.qubit', wires=2, shots=None)

In [ ]:
###
### YOUR CODE HERE
###

@qml.qnode(dev)
def circuit():
  qml.PauliX(wires=0)
  qml.Hadamard(wires=0)
  qml.CNOT(wires=[0,1])
  return qml.expval(qml.PauliZ(0)), qml.expval(qml.PauliZ(1)), qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))

In [ ]:
print(np.abs(circuit()-np.array([0,0,1]))<eps)
print(compare_states(dev.state,np.array([1,0,0,-1])/np.sqrt(2.)))

## Measurement

### Exercise 1
The measurements in current NISQ devices are performed in the computational basis. After a measurement is performed the state continues to be in a state that corresponding the the projected basis state. Write a function `circuit_x` and `circuit_z` that first create a superposition state $|\psi⟩=\frac{1}{\sqrt{2}}(|0⟩+|1⟩)$ and then use the method `qml.sample` in order return samples of the observables $A=\sigma^{\rm x}$ and $A=\sigma^{\rm z}$, respectively. Sampling in PennyLane is performed in the eigenbasis of the specified observable $A$. This means that the results are eigenvalues of the observable $\lambda_i$ with the probability $p_i=|⟨\xi_i|\psi⟩|^2$, where $A = \sum_{i}\lambda_i|\xi_i⟩⟨\xi_i|$.

In [ ]:
# DEVICE
dev = qml.device('default.qubit', wires=1, shots=100)

In [ ]:
###
### YOUR CODE HERE
###

@qml.qnode(dev)
def circuit_x():
    qml.RY(np.pi/2, wires=0)
    return qml.sample(qml.PauliX(wires=0))

@qml.qnode(dev)
def circuit_z():
    qml.RY(np.pi/2, wires=0)
    return qml.sample(qml.PauliZ(wires=0))

In [ ]:
print(np.mean([circuit_x() for _ in range(100)])==1.)
print(np.mean([circuit_z()  for _ in range(100)])<0.5)

**Note:** The output in the $x$ direction is deterministic, whereas the outpu in the $z$ direction is completely random. For any pure quantum state there is always an observable with a deterministic outcome and an observable which is completely random.

### Exercise 2
Write a function `circuit_zz` that prepares a two qubit entangled state $|\phi^+⟩=\frac{1}{\sqrt{2}}(|00⟩+|11⟩)$ and returns samples from observables $A_1 = \sigma^{\rm z}\otimes\mathbb{I}$ and $A_2 = \mathbb{I}⊗\sigma^{\rm z}$. Write also a function `circuit_zx` that also prepares the state $|\phi^+⟩$ and returns the samples from observables $A_1 = \sigma^{\rm z}\otimes\mathbb{I}$ and $A_2 = \mathbb{I}⊗\sigma^{\rm x}$.

In [ ]:
# DEVICE
dev = qml.device('default.qubit', wires=2, shots=100)

In [ ]:
@qml.qnode(dev)
def circuit_zz():
    qml.RY(np.pi/2, wires=0)
    qml.CNOT(wires=[0, 1])
    return qml.sample(qml.PauliZ(wires=0)), qml.sample(qml.PauliZ(wires=1))

@qml.qnode(dev)
def circuit_zx():
    qml.RY(np.pi/2, wires=0)
    qml.CNOT(wires=[0, 1])
    return qml.sample(qml.PauliZ(wires=0)), qml.sample(qml.PauliX(wires=1))

In [ ]:
samples=circuit_zz()
print(np.mean(samples[0]==samples[1])==1.0)

samples=circuit_zx()
print(np.abs(np.mean(samples[0]==samples[1])-0.5)<0.1)

**Note:** Samples in the $zz$ case are completely correlated since the state is entangled. However, if we choose a one observable in the $z$ direction and the other in the $x$ direction the samples become again completely uncorrelated, i.e. approx. 50\% of the time they agree.

### Exercise 3
Informationally complete measurement (ICM) is a measurement that uniquly determines the quantum probability. Create a set of functions `circuit1`, `circuit2`, `circuit3` that first prepare a state with the `hidden_preparation` template and then return one of the informatilnally complete measurement expectation values. Then wirte a function `reconstruct_hidden_state` that accepts the values of the ICM and returns a vector representing the original state. You can assume that the hidden state is pure.

In [ ]:
# DEVICE
dev = qml.device('default.qubit', wires=1, shots=None)

In [ ]:
from friqml.exercises.measurement import hidden_preparation, check_hidden_state

def e3_circuit1():
    hidden_preparation(wires=0)
    return qml.expval(qml.PauliX(0))

def e3_circuit2():
    hidden_preparation(wires=0)
    return qml.expval(qml.PauliY(0))

def e3_circuit3():
    hidden_preparation(wires=0)
    return qml.expval(qml.PauliZ(0))

@qml.qnode(dev)
def reconstruct_hidden_state(dev):
    circuit1 = qml.qnode(dev)(e3_circuit1)
    circuit2 = qml.qnode(dev)(e3_circuit2)
    circuit3 = qml.qnode(dev)(e3_circuit3)
    mx = circuit1()
    my = circuit2()
    mz = circuit3()

    phi = np.arctan2(my, mx)
    theta = np.arctan2(np.sqrt(mx**2+my**2), mz)
    psi = np.array([np.cos(theta/2), np.exp(-1j*phi)*np.sin(theta/2)])
    return psi

In [ ]:
#@qml.qnode(dev)
#def circuit(phi, theta):
#    qml.RY(theta, wires=0)
#    qml.RZ(phi, wires=0)
#    return qml.expval(qml.PauliX(0))

In [ ]:
check_hidden_state(reconstruct_hidden_state())

**Note:** Since product states are very unlikely a random state will be entangled with probability $1-\epsilon$.

## Noisy operations

### Exercise 1
Quantum devices are noisy. Noise sources can be divided into coherent and incoherent noise. Incoherent noise is modelled by quantum channels. One simple and important channel is a bit flip channel, that can be described by Kraus operators
\begin{align}
K_0 = \sqrt{1-p}\begin{pmatrix}1 & 0\\ 0 & 1\end{pmatrix}, \quad K_1 &= \sqrt{p}\begin{pmatrix}0 & 1\\ 1 & 0\end{pmatrix}.
\end{align}
This channel is implemented in PennyLane using the
`qml.BitFlip` operation. Create a function `circuit` with one argument `p` that creates an entangled state $|\psi\rangle$ and then simulates a noisy bit flip with the specified probability `p`. Finally the function should return the expectation value of the observable $A_{12}=\sigma^{\rm z}\otimes\sigma^{\rm z}$.

In [ ]:
# DEVICE
# In order to have support for mixed noisy operations
# we have to use the default.mixed device
dev = qml.device('default.mixed', wires=2)

In [ ]:
@qml.qnode(dev)
def circuit(p):
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    qml.BitFlip(p, wires=0)
    qml.BitFlip(p, wires=1)
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))

In [ ]:
ps = [0.001, 0.01, 0.1, 0.2]
evals=[0.996,0.9604,0.64,0.36]
for p,e in zip(ps,evals):
    print(np.round(circuit(p),4)==e)

### Exercise 2
Noise destroys quantum effects like interference. If we have a circuit with two Haddamard operations we get back the initial state. However if a depolarising noise is added in between the interference is reduced. Write a function `circuit` with an argument `p` which applies two Haddamard gates and between a depolarising noise with `qml.DepolarizingChannel(p, wires=0)`. Retrun the probabilities for the computational basis states.

Depolarising noise is given by Kraus operators
\begin{align}
K_0 &= \sqrt{1-p}\begin{pmatrix}1 & 0\\ 0 & 1\end{pmatrix}, &
K_1 &= \sqrt{p/3}\begin{pmatrix}0 & 1\\ 1 & 0\end{pmatrix}, \\
K_2 &= \sqrt{p/3}\begin{pmatrix}0 & -i\\ i & 0\end{pmatrix}, &
K_3 &= \sqrt{p/3}\begin{pmatrix}1 & 0\\ 0 & -1\end{pmatrix}.
\end{align}
What would happen if we would use `BitFlip` noise instead?

In [ ]:
# DEVICE
# In order to have support for mixed noisy operations
# we have to use the default.mixed device
dev = qml.device('default.mixed', wires=1)

In [ ]:
@qml.qnode(dev)
def circuit(p):
    qml.Hadamard(wires=0)
    qml.DepolarizingChannel(p, wires=0)
    qml.Hadamard(wires=0)
    return qml.probs(wires=[0])

@qml.qnode(dev)
def circuit_bf(p):
    qml.Hadamard(wires=0)
    qml.BitFlip(p, wires=0)
    qml.Hadamard(wires=0)
    return qml.probs(wires=[0])

In [ ]:
print(abs(circuit(0.3)[0]-0.8)<eps)
print(abs(circuit_bf(0.4)[0]-1)<eps)

## Entanglement game

Let us consider the following game played by two players; player A and player B. Each of the players has a fair coin. First, they flip the coin and then they send the result of the coin flip and a number ($a$ and $b$ respectively) -1 or 1 to the referee. The referee multiplies the numbers obtained from the players. The players win the game if the multiplied numbers satisfy the following table based on the coin flips

| Player A     | Player B  | Win ($a\cdot b$) |
|--------------|-----------|------------------|
|     0        | 0         | -1                |
|     0        | 1         | 1                |
|     1        | 0         | 1                |
|     1        | 1         | 1               |

The players can not communicate inbetween the game but can use all other resources and can decide on a strategy how they will play the game. The best classical strategy has a winning probability of 75%. Assuming the players share an entangled quantum state the best quantum strategy has a higher winning probability. The best quantum strategy is as follows. The player A performs two different sets of measurements, which is determined based on the result of the coin flip. If the result of the coin flip is 0 the player performs the measurement determined by the projectors {$|\psi(0)⟩⟨\psi(0)|$, $|\psi(\pi)⟩⟨\psi(\pi)|$}. If the result of the coin flip is 1 the player A performs the measurement {$|\psi(\pi/2)⟩⟨\psi(\pi/2)|$, $|\psi(-\pi/2)⟩⟨-\psi(\pi/2)|$}. Similarly the player B performs the measurement {$|\psi(\pi/4)⟩⟨\psi(\pi/4)|$, $|\psi(5\pi/4)⟩⟨\psi(5\pi/4)|$} if his coin is 1 and the measurement {$|\psi(3\pi/4)⟩⟨\psi(3\pi/4)|$, $|\psi(-\pi/4)⟩⟨\psi(-\pi/4)|$} if his coind is 0. For all measurements if the first state is observed the players return 1 to the referee otherwise they return -1. The projectors are defined by the state on the Bloch sphere

\begin{align}
  |\psi(\theta)\rangle &=\cos\theta/2|0⟩+\sin\theta/2|1⟩ \\
\end{align}

Write a function `circuit` that gets as arguments the results of the coin flips and then returns a sample from the correct observable. Use that function to determine the winning probability of the quantum strategy.


In [ ]:
# DEVICE
dev = qml.device('default.qubit', wires=2, shots=1)

In [ ]:
@qml.qnode(dev)
def entangled_phi():
    qml.RY(np.pi/2, wires=0)
    qml.CNOT(wires=[0, 1])

@qml.qnode(dev)
def circuit00():
    entangled_phi()
    qml.RY(-3*np.pi/4,wires=1)
    return qml.sample(qml.PauliZ(0) @ qml.PauliZ(1))

@qml.qnode(dev)
def circuit01():
    entangled_phi()
    qml.RY(np.pi/4,wires=1)
    return qml.sample(qml.PauliZ(0) @ qml.PauliX(1))

@qml.qnode(dev)
def circuit10():
    entangled_phi()
    qml.RY(-3*np.pi/4,wires=1)
    return qml.sample(qml.PauliX(0) @ qml.PauliZ(1))

@qml.qnode(dev)
def circuit11():
    entangled_phi()
    qml.RY(np.pi/4,wires=1)
    return qml.sample(qml.PauliX(0) @ qml.PauliX(1)) 

In [ ]:
def circuit(a,b):
  if a==0 and b == 0:
    return circuit00()
  if a==1 and b == 0:
    return circuit01()
  if a==0 and b == 1:
    return circuit10()
  if a==1 and b == 1:
    return circuit11()

def win(a,b):
  if a==0 and b==0:
    return -1
  return 1

def one_game():
  a = np.random.randint(2)
  b = np.random.randint(2)
  sab = circuit(a,b)
  return sab ==  win(a,b)

def quantum_strategy_probability(n):
  res = [one_game() for i in range(n)]
  return np.mean(res)

In [ ]:
print(f"Win rate of the best quantum strategy is {quantum_strategy_probability(1000)}.")
quantum_strategy_probability(1000)>0.75